In [31]:
import requests
import pandas as pd
import time
import json


In [32]:
headers={
    'accept': 'application/json',
    'x-api-key':''
}

In [33]:
def get_collections():
    url = f'https://api.opensea.io/api/v2/collections'
    params = {
        'order_by': 'seven_day_volume',
        'limit': 50,
    }
    next_page = False
    data = []
    loop =0
    while loop<=1:
        try:
            if next_page:
                params['next'] = next_page
            response = requests.get(url, params=params, headers=headers)
            response.raise_for_status()
            collections = response.json().get('collections', [])
            if collections:
                for collection in collections:
                    data.append({
                        'collection': collection.get('collection', ''),
                        'name': collection.get('name', ''),
                        'owner': collection.get('owner', ''),
                        'contracts_address': collection['contracts'][0].get('address', '') if 'contracts' in collection and collection['contracts'] else '',
                        'chain': 'ethereum'
                    })
            next_page = response.json().get('next', None)
            if not next_page:
                break
        except requests.exceptions.RequestException as e:
            print('Error:', e)
            break
        loop =loop+1

    return data

def get_all_events_by_collection(collection, event_type='sale'):
    base_url = f'https://api.opensea.io/api/v2/events/collection/{collection}'
    params = {'event_type': event_type,
              'after':1717200000,
              'before':1719705600}
    events_data = []
    next_page = False
    loop=0
    while True:
        try:
            if next_page:
                params['next'] = next_page
            response = requests.get(base_url, params=params, headers=headers)
            response.raise_for_status()
            json_data = response.json()
            events = json_data.get('asset_events', [])
            
            if not events:
                print(f'No {event_type} events found for collection {collection}')
                break
            for event in events:
                event_details = {
                    'collection': collection,
                    'event_type': event.get('event_type'),
                    'chain': event.get('chain'),
                    'tokenId': event.get('nft', {}).get('identifier') if isinstance(event.get('nft'), dict) else None,
                    'seller': event.get('seller'),
                    'buyer': event.get('buyer'),
                    'quantity': event.get('quantity'),
                    'price': event.get('payment', {}).get('quantity') if isinstance(event.get('payment'), dict) else None,
                    'decimals': event.get('payment', {}).get('decimals') if isinstance(event.get('payment'), dict) else None,
                    'symbol': event.get('payment', {}).get('symbol') if isinstance(event.get('payment'), dict) else None,
                    'transaction': event.get('transaction'),
                    'event_timestamp': event.get('event_timestamp')
                }
                events_data.append(event_details)
            next_page = json_data.get('next')
            if not next_page:
                break
            
        except requests.RequestException as e:
            print(f'Error fetching data: {e}')
            time.sleep(5)
            continue
        loop = loop+1
        print(loop)

    return events_data

def get_NFTs_by_collection(collection_slug):
    url =f'https://api.opensea.io/api/v2/collection/{collection_slug}/nfts'
    params={
        'limit': 200
    }
    NFTs = []
    next_page = False
    while True:
        try: 
            if next_page:
                params['next'] = next_page              
            response = requests.get(url, params=params,headers = headers)
            json_data = response.json()
            items = json_data.get('nfts','')
        
            if not items:
                break
            for item in items:
                NFTs.append({
                    "identifier": item.get('identifier'),
                    'collection': item.get('collection'),
                    'name': item.get('name'),
                    'update_at': item.get('updated_at'),
                })
            next_page = json_data.get('next',None)
            if next_page is None:
                break

        except requests.RequestException as e:
            print(f'Error fetching data: {e}')
            time.sleep(5)
            continue
        time.sleep(2)
    return NFTs

def get_NFT(token_id, ):

    url = f'https://api.opensea.io/api/v2/chain/ethereum/contract/0xbd3531da5cf5857e7cfaa92426877b022e612cf8/nfts/{token_id}'
   
    nft = []
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  
        data = response.json()

        if 'nft' in data:
            result = data['nft']
            owners = result.get('owners', [])
            owner_info = owners[0] if owners else {}

            nft.append({
                'identifier': result.get('identifier', ''),
                'owners_address': owner_info.get('address', ''),
                'quantity': owner_info.get('quantity', None)
            })

    except requests.exceptions.RequestException as e:  
        print(f'Error fetching data: {e}')
        time.sleep(60)

    return nft


In [34]:
# collections_data = get_collections()
# collections_data = pd.DataFrame(collections_data)

In [35]:
# collections_data.to_json('collections.json', orient ='records', indent =3)

In [36]:
# lấy event
# data = get_all_events_by_collection('pudgypenguins')
    

In [37]:
# data = pd.DataFrame(data)
# data.to_json('data_collection.json',orient = 'records', indent =3)

In [38]:
# nfts = get_NFTs_by_collection('pudgypenguins')
# nfts = pd.DataFrame(nfts)
# nfts.to_json('nfts.json', orient = 'records', indent =3)

In [39]:
with open('nfts.json', 'r') as file:
    nfts = json.load(file)
    
nfts = pd.DataFrame(nfts)

In [ ]:
nft_information = []
nft_erro =[]
loop = 0
for index, token_id in nfts.loc[1000:2000, 'identifier'].items():
    if loop == 50:
        time.sleep(60)
        loop = 0
    
    token_id = int(token_id)
    print(f"Index: {index}, Token ID: {token_id}")
    # Here you would call your get_NFT function
    nft = get_NFT(token_id)
    if nft is None:
        nft_erro.append({
            'index': index,
            'token_id': token_id
        })
        continue
    else:
        nft_information = nft_information + nft
    
    loop += 1
    
nft_information = pd.DataFrame(nft_information)
nft_information.to_json("nft_infor.json", orient ='records', indent =4)


In [ ]:
with open('nft_info.json', 'r') as file:
    nft_infor = json.load(file)

# Convert to DataFrame
nft_infor = pd.DataFrame(nft_infor)
print(len(nft_infor))

# Load nfts.json
with open('nfts.json', 'r') as file:
    nfts = json.load(file)

# Convert to DataFrame
nfts = pd.DataFrame(nfts)
print(len(nfts))

# Merge DataFrames on 'identity' column
nft = pd.merge(nft_infor, nfts, on='identifier')
print(len(nft))


In [156]:
nft.to_json('nfts_infor.json', orient= 'records', indent= 4)